In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

# --- 1. Load and Preprocess the Data ---

# Load the MNIST dataset
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values from 0-255 to 0-1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten the 28x28 images into 784-dimensional vectors
original_dim = 784
x_train = x_train.reshape((len(x_train), original_dim))
x_test = x_test.reshape((len(x_test), original_dim))

# --- 2. Create Noisy Data (The Corruption Process) ---

noise_factor = 0.5  # How much noise to add

# Create noisy training data by adding Gaussian noise
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
# Create noisy test data
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# Clip the values to stay in the [0, 1] range
x_train_noisy = np.clip(x_train_noisy, 0.0, 1.0)
x_test_noisy = np.clip(x_test_noisy, 0.0, 1.0)


# --- 3. Define the Autoencoder Architecture ---

# This is the size of our compressed code (the bottleneck)
encoding_dim = 32

# Input placeholder for the *noisy* images
input_img = Input(shape=(original_dim,))

# --- Encoder ---
# Compresses the noisy input
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# --- Decoder ---
# Reconstructs the *clean* image from the code
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(original_dim, activation='sigmoid')(decoded) # 'sigmoid' for [0, 1] pixels

# This model maps a NOISY input to its CLEAN reconstruction
autoencoder = Model(input_img, decoded)

# --- 4. Compile the Model ---

# We train to minimize the mean squared error
# between the *clean* image (x) and the reconstruction (r)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.summary()

# --- 5. Train the Model ---

print("Training the Denoising Autoencoder...")

# *** THE KEY STEP ***
# Input: x_train_noisy (corrupted images)
# Target: x_train (original, clean images)
autoencoder.fit(x_train_noisy, x_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

# --- 6. Visualize the Results ---

# Predict on the noisy test set
decoded_imgs = autoencoder.predict(x_test_noisy)

# Set up the plot
n = 10  # Number of digits to display
plt.figure(figsize=(20, 6))
plt.suptitle("Denoising Autoencoder Results", fontsize=16)

for i in range(n):
    # --- Display original images (for comparison) ---
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == 0:
        ax.set_title("Original Clean\nImages", fontsize=12)

    # --- Display noisy input images ---
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == 0:
        ax.set_title("Noisy Input", fontsize=12)

    # --- Display reconstructed (denoised) images ---
    ax = plt.subplot(3, n, i + 1 + 2 * n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == 0:
        ax.set_title("Denoised Output", fontsize=12)

plt.show()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 784)                 │         101,136 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 222,384 (868.69 KB)

 Trainable params: 222,384 (868.69 KB)

 Non-trainable params: 0 (0.00 B)

Training the Denoising Autoencoder...
Epoch 1/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - loss: 0.0951 - val_loss: 0.0492
Epoch 2/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 0.0450 - val_loss: 0.0368
Epoch 3/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 0.0356 - val_loss: 0.0322
Epoch 4/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 0.0320 - val_loss: 0.0300
Epoch 5/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - loss: 0.0299 - val_loss: 0.0284
Epoch 6/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - loss: 0.0283 - val_loss: 0.0269
Epoch 7/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - loss: 0.0271 - val_loss: 0.0261
Epoch 8/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.0260 - val_loss: 0.0251
Epoch 9/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - loss: 0.0251 - val_loss: 0.0245
Epoch 10/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - loss: 0.0244 - val_loss: 0.0238
Epoch 11/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 0.0239 - val_loss: 0.0233
Ep